# Multiscale Analysis of the Sustainable Production of 1,3PDO and 3HP
## X3 - Sensitivity Analysis: Purification Efficiency

In [72]:
%reset -f
%pylab inline

import cPickle as pickle
# from collections import OrderedDict
from cbmTES.global_constants import *
from framed.hack.ratio_constraint import *
import cbmTES.biochemical as biochemical
from framed.analysis.simulation import FBA, pFBA



Populating the interactive namespace from numpy and matplotlib


In [73]:

performances = pickle.load(open('jar/performances_gram.pickle','rb')) 

for p in performances:
    print p

('ecoli', 1, 'R_EX_glc_e', 'R_EX_13PDO_e')
('ecoli', 1, 'R_EX_glc_e', 'R_EX_3HPP_e')
('ecoli', 1, 'R_EX_glyc_e', 'R_EX_13PDO_e')
('ecoli', 1, 'R_EX_glyc_e', 'R_EX_3HPP_e')
('ecoli', 0, 'R_EX_glc_e', 'R_EX_13PDO_e')
('ecoli', 0, 'R_EX_glc_e', 'R_EX_3HPP_e')
('scere', 1, 'R_EX_glc_e', 'R_EX_13PDO_e')
('scere', 1, 'R_EX_glc_e', 'R_EX_3HPP_e')
('scere', 1, 'R_EX_glyc_e', 'R_EX_13PDO_e')
('scere', 1, 'R_EX_glyc_e', 'R_EX_3HPP_e')
('scere', 0, 'R_EX_glc_e', 'R_EX_13PDO_e')
('scere', 0, 'R_EX_glc_e', 'R_EX_3HPP_e')


In [74]:
# import the economy scale model
model = pickle.load(open('models/mini_BChI_model.pickle','rb'))



In [75]:
## Setting up scenario 1A, B, C, D

### Objective Functions ###
OBJs=OrderedDict()
MAX = True
MIN = False
OBJs['min_global_warming'] = ('EX_co2e_mkt', MIN) # minimize total global warming potential

### Flux Constraints ###
FCs = OrderedDict()
FCs['no import'] = {'DM_glyc_mkt':(0,0), 'DM_glc_mkt':(0,0), 'DM_energy_mkt':(0,0)}
FCs['product demand: 3hp = 1000']= {'DM_3hp_mkt':(1000,1000), 'DM_1,3pdo_mkt':(0,0)}
FCs['product demand: pdo = 1000']= {'DM_3hp_mkt':(0, 0), 'DM_1,3pdo_mkt':(1000,1000)}
FCs['product demand: 3hp = 500, pdo=500']= {'DM_3hp_mkt':(500,500), 'DM_1,3pdo_mkt':(500,500)}

FCs['positive profits']= {'PROFIT_res':(0,None), 'PROFIT_agr':(0,None), 'PROFIT_erg':(0,None), 'PROFIT_bch':(0,None)}
FCs['no biodiesel'] = {'energy_from_biodiesel':(0,0)}
FCs['land use < 1'] = {'EX_land_res':(-1,0)}
FCs['land use < 5'] = {'EX_land_res':(-5,0)}
FCs['co2 < 2000'] = {'EX_co2e_mkt':(0, 2000)}


### Ratio Constraints ###

# EU Energy Mix
def add_RC_EU_energy(model):
    add_ratio_constraint(model, 'energy_from_petroleum', 'MX_erg2mkt_energy', 0.25)
    add_ratio_constraint(model, 'energy_from_natural_gas', 'MX_erg2mkt_energy', 0.7)
    add_ratio_constraint(model, 'energy_from_coal', 'MX_erg2mkt_energy', 0.05)

def remove_RC_EU_energy(model):
    remove_ratio_constraint(model, 'ratio_energy_from_petroleum_MX_erg2mkt_energy')
    remove_ratio_constraint(model, 'ratio_energy_from_natural_gas_MX_erg2mkt_energy')
    remove_ratio_constraint(model, 'ratio_energy_from_coal_MX_erg2mkt_energy')

Scenarios = OrderedDict()
base_scenario = {k:v for d in (FCs['no import'], FCs['positive profits']) for k, v in d.iteritems()}
Scenarios['pdo1000, land<1, energy=EU'] = {k:v for d in (base_scenario, FCs['product demand: pdo = 1000'], FCs['land use < 1'], FCs['no biodiesel']) for k, v in d.iteritems()}


### Objective Functions ###
OBJs=OrderedDict()
MAX = True
MIN = False
OBJs['min_global_warming'] = ('EX_co2e_mkt', MIN) # minimize total global warming potential
OBJs['min_eutrofication'] = ('EX_po4e_mkt', MIN) # minimize total eutrophication potential
OBJs['max_bch_profit'] = ('PROFIT_bch', MAX) # maximize profit of biochemical industry
OBJs['min_bch_energy'] = ('MX_bch2mkt_energy', MAX) # minimize energy usage of biochemical industry

S = 'pdo1000, land<1, energy=EU'

if 'energy=EU' in S:
    add_RC_EU_energy(model)  # add EU energy mixing constraint




In [76]:
# sensitivity analysis
solutions = OrderedDict()
        
# Adding Bioprocesses to the model
for pe in [0.8, 0.9, 1]: # looping through purification efficiencies

    # remove the existing BioProcess Reactions 
    for rxn in model.reactions:
        if 'BPR' in rxn:
            model.remove_reaction(rxn)
    
    for key in performances.keys():
        strains = performances[key]
        #strains = biochemical.get_strains_metric(metric)

        if key[1] == AEROBIC:
            bioreactor = 'aeroFB'
        elif key[1] == ANAEROBIC:
            bioreactor = 'anaeFB'

        if key[2] == r_glc:
            substrate = 'glc'
        elif key[2] == r_glyc:
            substrate = 'glyc'

        if key[3] == r_13PDO:
            product = '1,3pdo'
        if key[3] == r_3HPP:
            product = '3hp'

        for i, strain in enumerate(strains):
            if strain['product_yield']>0 and strain['biomass_yield']>0:
                if product == '3hp':
                    biochemical.add_bioprocess_reaction(model, substrate, product, strain, bioreactor, 'ctrf', 'evapM', pe, neutralization='3hp_lime', process_energy_intensity=biochemical.Ei, compartment='bch')
                else:    
                    biochemical.add_bioprocess_reaction(model, substrate, product, strain, bioreactor, 'ctrf', 'evapM', pe, process_energy_intensity=biochemical.Ei, compartment='bch')

    for O in OBJs:
        print O
        solutions[O, S, pe] = FBA(model, objective={OBJs[O][0]:1}, maximize=OBJs[O][1], constraints = Scenarios[S])




min_global_warming
min_eutrofication
max_bch_profit
min_bch_energy
min_global_warming
min_eutrofication
max_bch_profit
min_bch_energy
min_global_warming
min_eutrofication
max_bch_profit
min_bch_energy


In [86]:
for (O, S, k) in solutions:
    print O, ' -- ', k,': ', solutions[O, S, k].fobj


min_global_warming  --  0.8 :  -2323.60566282
min_eutrofication  --  0.8 :  -12.2658500699
max_bch_profit  --  0.8 :  515.723598864
min_bch_energy  --  0.8 :  -12570.6992197
min_global_warming  --  0.9 :  -2065.42725584
min_eutrofication  --  0.9 :  -10.9029778399
max_bch_profit  --  0.9 :  659.678162366
min_bch_energy  --  0.9 :  -11136.4427421
min_global_warming  --  1 :  -1858.88453026
min_eutrofication  --  1 :  -9.81268005595
max_bch_profit  --  1 :  774.841813167
min_bch_energy  --  1 :  -10022.7984679
